In [1]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import torch.nn as nn
from torchvision import transforms
import torchvision
import random
from tqdm import tqdm
from PIL import Image

In [2]:
def WaveletTransformAxisY(batch_img):
    odd_img  = batch_img[:,0::2]
    even_img = batch_img[:,1::2]
    L = (odd_img + even_img) / 2.0
    H = torch.abs(odd_img - even_img)

    return L, H

In [3]:
def WaveletTransformAxisX(batch_img):
    # transpose + fliplr
    tmp_batch = torch.permute(batch_img, (0, 2, 1))
    tmp_batch = torch.fliplr(tmp_batch)
    _dst_L, _dst_H = WaveletTransformAxisY(tmp_batch)
    # transpose + flipud
    dst_L = torch.permute(_dst_L, [0, 2, 1])
    dst_L = torch.flipud(dst_L)
    dst_H = torch.permute(_dst_H, [0, 2, 1])
    dst_H = torch.flipud(dst_H)

    return dst_L, dst_H

In [4]:
def Wavelet(batch_image):
    r = batch_image[:,0]
    g = batch_image[:,1]
    b = batch_image[:,2]

    # level 1 decomposition
    wavelet_L, wavelet_H = WaveletTransformAxisY(r)
    r_wavelet_LL, r_wavelet_LH = WaveletTransformAxisX(wavelet_L)
    r_wavelet_HL, r_wavelet_HH = WaveletTransformAxisX(wavelet_H)

    wavelet_L, wavelet_H = WaveletTransformAxisY(g)
    g_wavelet_LL, g_wavelet_LH = WaveletTransformAxisX(wavelet_L)
    g_wavelet_HL, g_wavelet_HH = WaveletTransformAxisX(wavelet_H)

    wavelet_L, wavelet_H = WaveletTransformAxisY(b)
    b_wavelet_LL, b_wavelet_LH = WaveletTransformAxisX(wavelet_L)
    b_wavelet_HL, b_wavelet_HH = WaveletTransformAxisX(wavelet_H)

    wavelet_data = [r_wavelet_LL, r_wavelet_LH, r_wavelet_HL, r_wavelet_HH,
                    g_wavelet_LL, g_wavelet_LH, g_wavelet_HL, g_wavelet_HH,
                    b_wavelet_LL, b_wavelet_LH, b_wavelet_HL, b_wavelet_HH]
    transform_batch = torch.stack(wavelet_data, axis=1)

    # level 2 decomposition
    wavelet_L2, wavelet_H2 = WaveletTransformAxisY(r_wavelet_LL)
    r_wavelet_LL2, r_wavelet_LH2 = WaveletTransformAxisX(wavelet_L2)
    r_wavelet_HL2, r_wavelet_HH2 = WaveletTransformAxisX(wavelet_H2)

    wavelet_L2, wavelet_H2 = WaveletTransformAxisY(g_wavelet_LL)
    g_wavelet_LL2, g_wavelet_LH2 = WaveletTransformAxisX(wavelet_L2)
    g_wavelet_HL2, g_wavelet_HH2 = WaveletTransformAxisX(wavelet_H2)

    wavelet_L2, wavelet_H2 = WaveletTransformAxisY(b_wavelet_LL)
    b_wavelet_LL2, b_wavelet_LH2 = WaveletTransformAxisX(wavelet_L2)
    b_wavelet_HL2, b_wavelet_HH2 = WaveletTransformAxisX(wavelet_H2)


    wavelet_data_l2 = [r_wavelet_LL2, r_wavelet_LH2, r_wavelet_HL2, r_wavelet_HH2,
                       g_wavelet_LL2, g_wavelet_LH2, g_wavelet_HL2, g_wavelet_HH2,
                       b_wavelet_LL2, b_wavelet_LH2, b_wavelet_HL2, b_wavelet_HH2]
    transform_batch_l2 = torch.stack(wavelet_data_l2, dim=1)

    # level 3 decomposition
    wavelet_L3, wavelet_H3 = WaveletTransformAxisY(r_wavelet_LL2)
    r_wavelet_LL3, r_wavelet_LH3 = WaveletTransformAxisX(wavelet_L3)
    r_wavelet_HL3, r_wavelet_HH3 = WaveletTransformAxisX(wavelet_H3)

    wavelet_L3, wavelet_H3 = WaveletTransformAxisY(g_wavelet_LL2)
    g_wavelet_LL3, g_wavelet_LH3 = WaveletTransformAxisX(wavelet_L3)
    g_wavelet_HL3, g_wavelet_HH3 = WaveletTransformAxisX(wavelet_H3)

    wavelet_L3, wavelet_H3 = WaveletTransformAxisY(b_wavelet_LL2)
    b_wavelet_LL3, b_wavelet_LH3 = WaveletTransformAxisX(wavelet_L3)
    b_wavelet_HL3, b_wavelet_HH3 = WaveletTransformAxisX(wavelet_H3)

    wavelet_data_l3 = [r_wavelet_LL3, r_wavelet_LH3, r_wavelet_HL3, r_wavelet_HH3,
                       g_wavelet_LL3, g_wavelet_LH3, g_wavelet_HL3, g_wavelet_HH3,
                       b_wavelet_LL3, b_wavelet_LH3, b_wavelet_HL3, b_wavelet_HH3]
    transform_batch_l3 = torch.stack(wavelet_data_l3, dim=1)

    # level 4 decomposition
    wavelet_L4, wavelet_H4 = WaveletTransformAxisY(r_wavelet_LL3)
    r_wavelet_LL4, r_wavelet_LH4 = WaveletTransformAxisX(wavelet_L4)
    r_wavelet_HL4, r_wavelet_HH4 = WaveletTransformAxisX(wavelet_H4)

    wavelet_L4, wavelet_H4 = WaveletTransformAxisY(g_wavelet_LL3)
    g_wavelet_LL4, g_wavelet_LH4 = WaveletTransformAxisX(wavelet_L4)
    g_wavelet_HL4, g_wavelet_HH4 = WaveletTransformAxisX(wavelet_H4)

    wavelet_L4, wavelet_H4 = WaveletTransformAxisY(b_wavelet_LL3)
    b_wavelet_LL4, b_wavelet_LH4 = WaveletTransformAxisX(wavelet_L4)
    b_wavelet_HL4, b_wavelet_HH4 = WaveletTransformAxisX(wavelet_H4)


    wavelet_data_l4 = [r_wavelet_LL4, r_wavelet_LH4, r_wavelet_HL4, r_wavelet_HH4,
                       g_wavelet_LL4, g_wavelet_LH4, g_wavelet_HL4, g_wavelet_HH4,
                       b_wavelet_LL4, b_wavelet_LH4, b_wavelet_HL4, b_wavelet_HH4]
    transform_batch_l4 = torch.stack(wavelet_data_l4, dim=1)

    return [transform_batch, transform_batch_l2, transform_batch_l3, transform_batch_l4]

In [5]:
class Wavelet_Model(torch.nn.Module):
    def __init__(self, classes=10):
        super(Wavelet_Model, self).__init__()
        self.conv_1 = nn.Conv2d(12, 64, kernel_size=(3, 3), padding=1)
        self.norm_1 = nn.BatchNorm2d(64)
        self.relu_1 = nn.ReLU()

        self.conv_1_2 = nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=1)
        self.norm_1_2 = nn.BatchNorm2d(64)
        self.relu_1_2 = nn.ReLU()
        #################################################################################################
        self.conv_a = nn.Conv2d(12, 64, kernel_size=(3, 3), padding=1)
        self.norm_a = nn.BatchNorm2d(64)
        self.relu_a = nn.ReLU()
        #################################################################################################
        self.conv_2 = nn.Conv2d(128, 128, kernel_size=(3, 3), padding=1)
        self.norm_2 = nn.BatchNorm2d(128)
        self.relu_2 = nn.ReLU()

        self.conv_2_2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=1)
        self.norm_2_2 = nn.BatchNorm2d(128)
        self.relu_2_2 = nn.ReLU()
        #################################################################################################
        self.conv_b = nn.Conv2d(12, 128, kernel_size=(3, 3), padding=1)
        self.norm_b = nn.BatchNorm2d(128)
        self.relu_b = nn.ReLU()

        self.conv_b_2 = nn.Conv2d(128, 128, kernel_size=(3, 3), padding=1)
        self.norm_b_2 = nn.BatchNorm2d(128)
        self.relu_b_2 = nn.ReLU()
        #################################################################################################
        self.conv_3 = nn.Conv2d(256, 256, kernel_size=(3, 3), padding=1)
        self.norm_3 = nn.BatchNorm2d(256)
        self.relu_3 = nn.ReLU()

        self.conv_3_2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=1)
        self.norm_3_2 = nn.BatchNorm2d(256)
        self.relu_3_2 = nn.ReLU()
        #################################################################################################
        self.conv_c = nn.Conv2d(12, 256, kernel_size=(3, 3), padding=1)
        self.norm_c = nn.BatchNorm2d(256)
        self.relu_c = nn.ReLU()

        self.conv_c_2 = nn.Conv2d(256, 256, kernel_size=(3, 3), padding=1)
        self.norm_c_2 = nn.BatchNorm2d(256)
        self.relu_c_2 = nn.ReLU()

        self.conv_c_3 = nn.Conv2d(256, 256, kernel_size=(3, 3), padding=1)
        self.norm_c_3 = nn.BatchNorm2d(256)
        self.relu_c_3 = nn.ReLU()
        #################################################################################################
        self.conv_4 = nn.Conv2d(512, 256, kernel_size=(3, 3), padding=1)
        self.norm_4 = nn.BatchNorm2d(256)
        self.relu_4 = nn.ReLU()

        self.conv_4_2 = nn.Conv2d(256, 128, kernel_size=(3, 3), stride=(2, 2), padding=1)
        self.norm_4_2 = nn.BatchNorm2d(128)
        self.relu_4_2 = nn.ReLU()
        #################################################################################################
        self.conv_5 = nn.Conv2d(128, 128, kernel_size=(3, 3), padding=1)
        self.norm_5 = nn.BatchNorm2d(128)
        self.relu_5 = nn.ReLU()

        self.pool_5 = nn.AvgPool2d(kernel_size=(7,7), stride=1, padding=1)
        self.flat_5 = nn.Flatten()

        self.fc_5 = nn.Linear(1152, 2048)
        self.norm_5_1 = nn.BatchNorm1d(2048)
        self.relu_5_1 = nn.ReLU()
        self.drop_5 = nn.Dropout(0.5)
        #################################################################################################
        self.fc_6 = nn.Linear(2048, classes)
        self.norm_6 = nn.BatchNorm1d(classes)
        self.relu_6 = nn.ReLU()
        self.drop_6 = nn.Dropout(0.5)
        #################################################################################################
        self.output_fc = nn.Linear(classes, classes)

    def forward(self, x):
        input_l1, input_l2, input_l3, input_l4 = Wavelet(x)
        #################################################################################################
        # print('input shape: ', input_l1.shape)
        out_1 = self.conv_1(input_l1)
        # print('conv_1 output shape: ', out_1.shape)
        out_1 = self.norm_1(out_1)
        out_1 = self.relu_1(out_1)

        out_1 = self.conv_1_2(out_1)
        # print('conv_1_2 output shape: ', out_1.shape)
        out_1 = self.norm_1_2(out_1)
        out_1 = self.relu_1_2(out_1)
        #################################################################################################
        out_2 = self.conv_a(input_l2)
        # print('conv_a output shape: ', out_2.shape)
        out_2 = self.norm_a(out_2)
        out_2 = self.relu_a(out_2)

        cat_2 = torch.cat((out_1, out_2), 1)
        # print('concatenate result: ', cat_2.shape)
        out_2 = self.conv_2(cat_2)
        # print('conv_2 output shape: ', out_2.shape)
        out_2 = self.norm_2(out_2)
        out_2 = self.relu_2(out_2)

        out_2 = self.conv_2_2(out_2)
        # print('conv_2_2 output shape: ', out_2.shape)
        out_2 = self.norm_2_2(out_2)
        out_2 = self.relu_2_2(out_2)
        #################################################################################################
        out_3 = self.conv_b(input_l3)
        # print('conv_b output shape: ', out_3.shape)
        out_3 = self.norm_b(out_3)
        out_3 = self.relu_b(out_3)

        out_3 = self.conv_b_2(out_3)
        # print('conv_b_2 output shape: ', out_3.shape)
        out_3 = self.norm_b_2(out_3)
        out_3 = self.relu_b_2(out_3)
        #################################################################################################
        cat_3 = torch.cat((out_2, out_3), 1)
        # print('concatenate result: ', cat_3.shape)
        out_3 = self.conv_3(cat_3)
        # print('conv_3 output shape: ', out_3.shape)
        out_3 = self.norm_3(out_3)
        out_3 = self.relu_3(out_3)

        out_3 = self.conv_3_2(out_3)
        # print('conv_3_2 output shape: ', out_3.shape)
        out_3 = self.norm_3_2(out_3)
        out_3 = self.relu_3_2(out_3)
        #################################################################################################
        out_4 = self.conv_c(input_l4)
        # print('conv_c output shape: ', out_4.shape)
        out_4 = self.norm_c(out_4)
        out_4 = self.relu_c(out_4)

        out_4 = self.conv_c_2(out_4)
        # print('conv_c_2 output shape: ', out_4.shape)
        out_4 = self.norm_c_2(out_4)
        out_4 = self.relu_c_2(out_4)

        out_4 = self.conv_c_3(out_4)
        # print('conv_c_3 output shape: ', out_4.shape)
        out_4 = self.norm_c_3(out_4)
        out_4 = self.relu_c_3(out_4)
        #################################################################################################
        cat_4 = torch.cat((out_3, out_4), 1)
        # print('concatenate result: ', cat_4.shape)
        out_4 = self.conv_4(cat_4)
        # print('conv_4 output shape: ', out_4.shape)
        out_4 = self.norm_4(out_4)
        out_4 = self.relu_4(out_4)

        out_4 = self.conv_4_2(out_4)
        # print('conv_4_2 output shape: ', out_4.shape)
        out_4 = self.norm_4_2(out_4)
        out_4 = self.relu_4_2(out_4)
        #################################################################################################
        out_5 = self.conv_5(out_4)
        # print('conv_5 output shape: ', out_5.shape)
        out_5 = self.norm_5(out_5)
        out_5 = self.relu_5(out_5)

        out_5 = self.pool_5(out_5)
        out_5 = self.flat_5(out_5)
        #################################################################################################
        out_5 = self.fc_5(out_5)
        # print('fc_5 output shape: ', out_5.shape)
        out_5 = self.norm_5_1(out_5)
        out_5 = self.relu_5_1(out_5)
        out_5 = self.drop_5(out_5)

        out_6 = self.fc_6(out_5)
        # print('fc6 output shape: ', out_6.shape)
        out_6 = self.norm_6(out_6)
        out_6 = self.relu_6(out_6)
        out_6 = self.drop_6(out_6)
        #################################################################################################
        output = self.output_fc(out_6)

        return output

In [6]:
import wandb

wandb.login()

wandb: Currently logged in as: mindw96 (use `wandb login --relogin` to force relogin)


True

In [7]:
batch_size  = 512
random_seed = 888
random.seed(random_seed)
torch.manual_seed(random_seed)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [23]:
config = dict(
    epochs=300,
    classes=58,
    batch_size=512,
    learning_rate=0.0005,
    dataset="KTH-TIPS2+DTD",
    architecture="Wavelet_CNN",
    data_path = './dataset/'
)

In [24]:
def model_pipeline(hyperparameters):

    # tell wandb to get started
    with wandb.init(project="Texture Classification", config=hyperparameters):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config

        # make the model, data, and optimization problem
        model, train_loader, test_loader, criterion, optimizer = make(config)

        # and use them to train the model
        train(model, train_loader, criterion, optimizer, config)

        # and test its final performance
        # skip test
        # test(model, test_loader)

    return model

In [25]:
def make(config):
    # Make the data
    datasets = get_data(config.data_path)
    train_loader = make_loader(datasets, batch_size=config.batch_size)
    # valid_loader = make_loader(datasets['valid'], batch_size=config.batch_size)

    # Make the model
    model = Wavelet_Model(config.classes).to(device)

    # Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config.learning_rate)

    return model, train_loader, criterion, optimizer

In [26]:
def get_data(data_path=''):
    texture_dataset = torchvision.datasets.ImageFolder(
        data_path,
        transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]))
    #  equiv to slicing with [::slice]
    # train_idx, val_idx = train_test_split(list(range(len(texture_dataset))), test_size=0.2, random_state=random_seed)
    # datasets = {}
    # datasets['train'] = Subset(texture_dataset, train_idx)
    # datasets['valid'] = Subset(texture_dataset, val_idx)

    return texture_dataset


def make_loader(dataset, batch_size):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         pin_memory=True, num_workers=20)
    return loader

In [27]:
def train(model, loader, criterion, optimizer, config):
    # tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
        for _, (images, labels) in enumerate(loader):

            loss, accuracy = train_batch(images, labels, model, optimizer, criterion)
            example_ct +=  len(images)
            batch_ct += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 25) == 0:
                train_log(loss, example_ct, epoch, accuracy)

    torch.onnx.export(model, images, "model.onnx")
    wandb.save("model.onnx")


def train_batch(images, labels, model, optimizer, criterion):
    images, labels = images.to(device), labels.to(device)
    correct, total = 0, 0
    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    accuracy = correct / total

    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss, accuracy

In [28]:
def train_log(loss, example_ct, epoch, accuracy):
    loss = float(loss)

    # where the magic happens
    wandb.log({"epoch": epoch, "loss": loss, "accuracy": accuracy}, step=example_ct)
    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [29]:
def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Accuracy of the model on the {total} " +
              f"test images: {100 * correct / total}%")

        wandb.log({"test_accuracy": correct / total})

    # Save the model in the exchangeable ONNX format
    torch.onnx.export(model, images, "model.onnx")
    wandb.save("model.onnx")

In [30]:
model = model_pipeline(config)

Wavelet_Model(
  (conv_1): Conv2d(12, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_1): ReLU()
  (conv_1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (norm_1_2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_1_2): ReLU()
  (conv_a): Conv2d(12, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm_a): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_a): ReLU()
  (conv_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm_2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_2): ReLU()
  (conv_2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (norm_2_2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_2_2): ReLU()
  (conv_b): Conv2d(12,

  0%|          | 1/300 [00:33<2:44:38, 33.04s/it]

Loss after 11897 examples: 3.280


  1%|          | 2/300 [01:05<2:41:46, 32.57s/it]

Loss after 24306 examples: 2.844


...

Loss after 2493779 examples: 0.036


100%|██████████| 300/300 [2:50:49<00:00, 34.16s/it]


Accuracy of the model on the 2079 test images: 65.56036556036555%


accuracy,▁▂▂▃▃▃▅▄▄▅▅▆▆▇▇▇▇▇▇▇█▇██████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▅▅▅▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁
accuracy,0.99414
epoch,299
loss,0.03633
test_accuracy,0.6556


RuntimeError: step!=1 is currently not supported

In [33]:
img = Image.open('./test_samples/cork.png')
tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()])
img_tensor = tf(img)
img_tensor = img_tensor.unsqueeze(0)
print(img_tensor.shape)

torch.Size([1, 3, 224, 224])


In [34]:
output = model(img_tensor)
_, predict = torch.max(output, 1)
score = torch.nn.functional.softmax(output, dim=1).detach().numpy()
img.show()


# print(
#     "This image most likely belongs to {} with a {:.2f} percent confidence."
#         .format(class_names[(np.argmax(score))], 100 * np.max(score))
# )

NameError: name 'model' is not defined

In [40]:
from IPython.display import IFrame

IFrame("https://wandb.ai/mindw96/TextureClassification/2pvdkac1", width="100%", height=720)